<img src="Images/Logo.png" alt="Logo NSI" style="float:right">

<h1 style="text-align:center">Chapitre 30 : Fusion de tables</h1>

Il est fréquent, lorsque l'on travaille avec des données, et particulièrement avec des données tabulées, de se retrouver avec plusieurs jeux de données.  
On peut alors souhaiter regrouper ces jeux de données en une seule table.  
Une fois ces données combinées en une seule table, nous pouvons réutiliser les opérations de filtrage, sélection de colonnes, agrégation et tri.

Il existe plusieurs façon de **fusionner** des données en tables.  
Selon l'opération que l'on souhaite effectuer, des précautions particulières sont à prendre pour ne pas introduire d'incohérences dans les données.

## Réunion de tables
Une première opération naturelle est de vouloir mettre dans une même table les données de deux tables existantes.  
Sur les [plate-formes de données publiques françaises](https://www.data.gouv.fr/), on peut trouver des fichiers CSV recensant les listes des prénoms d'enfants nés dans certaines villes. Chaque année est stockée dans un fichier propre : `prenoms2016.csv`, `prenoms2017.csv`,  [`prenoms2018.csv`](Donnees/prenoms2018.csv), [`prenoms2019.csv`](Donnees/prenoms2019.csv), ...

Il est naturel de vouloir réunir tous ces jeux de données en un seul, pour former une seule grande table que l'on peut ensuite filtrer ou trier.  
Cette opération a du sens car tous ces fichiers, et donc les tables correspondantes, possèdent la même structure : elles ont les mêmes attributs (en nom et en nombre) et les attributs de même nom sont du même type.  
Pour notre exemple, ces attributs donnent l'année de naissance, le code de la commune, le nom de la commune, le sexe, le prénom et le nombre d'enfants nés avec ce prénom.

| Année de naissance | Code commune | Libellé commune | Sexe     | Prénom            | Nombre |
|:-------------------:|:-------------:|:----------------:|:---------:|:------------------:|:-------:|
| 2018               | 35238        | Rennes          | Masculin | Abdoulmalik       | 1      |
|...|...|...|...|...|...|

Chargeons les données des années 2018 et 2019, contenues dans les deux fichiers [`prenoms2018.csv`](Donnees/prenoms2018.csv) et [`prenoms2019.csv`](Donnees/prenoms2019.csv).

In [ ]:
import csv

with open("Donnees/prenoms2018.csv") as fichier2018:
    lecteur = csv.DictReader(fichier2018, delimiter=';')
    table2018 = [{attribut: ligne[attribut] for attribut in ligne} for ligne in lecteur]

with open("Donnees/prenoms2019.csv") as fichier2019:
    lecteur = csv.DictReader(fichier2019, delimiter=';')
    table2019 = [{attribut: ligne[attribut] for attribut in ligne} for ligne in lecteur]

Les deux tables ayant les mêmes attributs, il est possible de les réunir en une seule table contenant toutes les valeurs.

In [ ]:
table2018_19 = table2018 + table2019

Le tableau `table2018_19` que l'on obtient est un nouveau tableau contenant les éléments de `table2018` suivi des éléments de `table2019`.  
Du point de vue de Python, cette opération est toujours autorisée entre deux tableaux quelconques.

Dans le cadre des tables de données, il est préférable de considérer les opérations sur les tables.  

Commençons par la sélection de lignes d'une table.  
Si l'on souhaite connaître les prénoms de toutes les petites filles nées en 2018, on peut écrire le code suivant :

In [ ]:
fille2018 = [ligne for ligne in table2018 if ligne["Sexe"] == "Féminin"]

On pourra donc répéter l'opération pour la table `table2019`, de même que sur la table `table2018_19`.

In [ ]:
fille2019 = [ligne for ligne in table2019 if ligne["Sexe"] == "Féminin"]
fille2018_19 = [ligne for ligne in table2018_19 if ligne["Sexe"] == "Féminin"]

Si nous avions réunis les tables `table2018` et [`eleves`](Donnees/eleves.csv), nous aurions un problème pour effectuer cette opération.

```python
mauvaise_table = table2018 + eleves
test = [ligne for ligne in mauvaise_table if ligne["Sexe"] == "Féminin"]
```
On obtient alors une erreur :

```python
KeyError: 'Sexe'
```

Réunir les valeurs de deux tables est une opération légitime, à condition que les tables aient les mêmes attributs et les mêmes domaines de valeurs pour leurs attributs.

## Opération de jointure
Nous pouvons aller plus loin et considérer des tables ayant des attributs différents, mais **au moins** un attribut commun.

Pour cela considérons la table [`eleves`](Donnees/eleves.csv)

In [ ]:
import csv

with open("Donnees/eleves.csv") as fichier:
    lecteur = csv.DictReader(fichier, delimiter=',')
    table_eleves = [{attribut: ligne[attribut] for attribut in ligne} for ligne in lecteur]
    
def valide_eleve(dico: dict[str, str]) -> dict[str, str | int]:
    prenom = dico["prénom"]
    jour = int(dico["jour"])
    mois = int(dico["mois"])
    annee = int(dico["année"])
    projet = dico["projet"]
    assert mois >= 1 and mois <= 12, "mois invalide dans le fichier CSV"
    return {"prénom": prenom, "jour": jour, "mois": mois, "année": annee, "projet": projet}

eleves = [valide_eleve(ligne) for ligne in table_eleves]

Parmi ces élèves, Brian, Linus et Blaise ont rendu leur projet.  
Le professeur renseigne les notes dans la table [`notes`](Donnees/notes.csv)

In [ ]:
with open("Donnees/notes.csv") as fichier:
    lecteur = csv.DictReader(fichier, delimiter=',')
    table_notes = [{attribut: ligne[attribut] for attribut in ligne} for ligne in lecteur]
    
def valide_note(dico: dict[str, str]) -> dict[str, str | float]:
    prenom = dico["prénom"]
    note = float(dico["note"])
    assert note >= 0 and note <= 20, "Note invalide dans le fichier CSV"
    return {"prénom": prenom, "note": note}

notes = [valide_note(ligne) for ligne in table_notes]

On souhaite maintenant éditer un relevé de notes, c'est-à-dire une table donnant, pour chaque élève qui a eu une note, son prénom, sa date de naissance, le nom du projet et sa note dans le projet.  
On commence par définir une fonction qui, étant donnée une ligne de la table `eleves` et une ligne de la table `notes`, créé un nouveau dictionnaire représentant la ligne du résultat

In [ ]:
def fusion(tab1: dict[str, str | int], tab2: dict[str, str | float]) -> dict[str, str | int | float]:
    return {"prénom": tab1["prénom"], "jour": tab1["jour"], 
            "mois": tab1["mois"], "année": tab1["année"],
            "projet": tab1["projet"], "note": tab2["note"]}

La clé `prénom` est présente dans les deux tables.  
L'algorithme de fusion de table est une double boucle

In [ ]:
tab = []
for el in eleves:
    for n in notes:
        if el["prénom"] == n["prénom"]:
            tab.append(fusion(el, n))

On peut également obtenir le même résultat en utilisant une *double compréhension* de tableaux.

In [ ]:
tab2 = [fusion(el, n) for el in eleves
        for n in notes if el["prénom"] == n["prénom"]]

Cette opération de mise en correspondance de lignes de deux tables ayant un attribut commun est appelé **jointure** de deux tables.  
C'est un outil très puissant dans la gestion des données tabulées.

Par exemple, si on considère nos tables des prénoms `table2018` et `table2019`, on peut calculer la table `evolution` contenant, pour chaque prénom, le nombre de naissances correspondantes en 2018 et 2019, ce qui permet de calculer la variation sur une année pour un prénom donné.

In [ ]:
evolution = [{"Prénom": ligne1["Prénom"], 
              "Nombre 2018": ligne1["Nombre"], 
              "Nombre 2019": ligne2["Nombre"]} for ligne1 in table2018
             for ligne2 in table2019 if ligne1["Prénom"] == ligne2["Prénom"]]

## Utilisation d'un identifiant unique
Lors de la jointure de tables nous avons fait une hypothèse implicite, à savoir que les prénoms étaient uniques.  
Cette hypothèse est vraie pour les fichiers [`prenoms2018.csv`](Donnees/prenoms2018.csv) et [`prenoms2019.csv`](Donnees/prenoms2019.csv) mais pas pour le fichier [`eleves.csv`](Donnees/eleves.csv) (il y a deux Alan).   
Il est donc important de pouvoir associer un identifiant unique à chaque ligne d'une table car les données elles-mêmes ne permettent pas forcément de distinguer deux entrées de la table.  
Une manière de faire est d'associer, au moment du chargement, un entier unique pour chaque entrée de le table.

In [ ]:
with open("Donnees/eleves.csv") as fichier:
    lecteur = csv.DictReader(fichier, delimiter=',')
    eleves = [{attribut: ligne[attribut] for attribut in ligne} for ligne in lecteur]
    
compteur = 0
for ligne in eleves:
    ligne["id"] = compteur
    compteur += 1

Ce code permet de modifier, **en place**, tous les enregistrements de la table chargée, en leur ajoutant un attribut `id` dont la valeur est un entier incrémenté à chaque ligne.  
Cet incrément permet d'assurer l'unicité de l'identifiant.  
On obtient la table suivante :

| id | prénom       | jour | mois | année | projet                                          |
|:----:|:--------------:|:------:|:------:|:-------:|:-------------------------------------------------:|
| 0  | Brian        | 1    | 1    | 1942  | programmer avec style                           |
| 1  | Grace        | 9    | 12   | 1906  | production automatique de code machine          |
| 2  | Linus        | 28   | 12   | 1969  | un petit système d'exploitation                 |
| 3  | Donald       | 10   | 1    | 1938  | tout sur les algorithmes                        |
| 4  | Alan         | 23   | 6    | 1912  | déchiffrer des codes secrets                    |
| 5  | Blaise       | 19   | 6    | 1623  | machine arithmétique                            |
| 6  | Margaret     | 17   | 8    | 1936  | assistance à l'atterrissage d'un module lunaire |
| 7  | Alan         | 1    | 4    | 1922  | tout ce qu'un programmeur devrait savoir        |
| 8  | Joseph Marie | 7    | 7    | 1752  | programmer des dessins                          |

Le professeur peut alors rendre, de manière non ambigüe, sa note à Alan, en le référençant par son `id`.

| id | note       |  
|:----:|:--------------:|
| 7  | 15        |

La jointure peut alors se faire sur l'attribut `id`.

In [ ]:
notes = [{"id": 7, "note": 15}]

def fusion(tab1: dict[str, str | int], tab2: dict[str, str | float]) -> dict[str, str | int | float]:
    return {"id": tab1["id"], "prénom": tab1["prénom"], "jour": tab1["jour"], 
            "mois": tab1["mois"], "année": tab1["année"],
            "projet": tab1["projet"], "note": tab2["note"]}

tab3 = [fusion(el, n) for el in eleves
        for n in notes if el["id"] == n["id"]]

## Utilisation de la bibliothèque [pandas](https://pandas.pydata.org/)

### Lecture et importation du fichier CSV

In [ ]:
import pandas

pays = pandas.read_csv("Donnees/countries.csv", delimiter=';', keep_default_na=False)
villes = pandas.read_csv("Donnees/cities.csv", delimiter=';')

### Fusion de tables
Dans la table des [`pays`](Donnees/countries.csv), la capitale est indiquée par un numéro qui correspond au champ `id` de la table des [`villes`](Donnees/cities.csv).  
Pour récupérer le nom de la capitale de chaque pays, nous allons fusionner les tables en effectuant une jointure. Ainsi, nous allons faire correspondre le champ `capital` de pays et le champ `id` de villes. Cela se fait à l’aide de la fonction [`merge`](https://pandas.pydata.org/pandas-docs/dev/reference/api/pandas.merge.html).

In [ ]:
pandas.merge(pays, villes, left_on="capital", right_on="id")

Cependant, en procédant ainsi, il va y avoir un conflit entre les champs des deux tables.  
On voit que des tables initiales contiennent toutes les deux des champs `name` et `population`, d’où les suffixes `_x` et `_y` pour marquer la référence à la première table initiale ou à la seconde.
Pour rendre cela plus lisible, nous allons :
1. ne garder que les colonnes de ville qui nous intéressent, ici l’identifiant et le nom
2. renommer ces colonnes pour éviter les collisions avec les champs de pays

In [ ]:
villes[["id", "name"]].rename(columns = {"id": "capital", "name": "capital_name"})

Et c’est cette nouvelle table que nous allons fusionner avec la table `pays` (dont nous ne garderons pas toutes les colonnes non plus) :

In [ ]:
pays_et_capitales = pandas.merge(pays[["iso", "name", "capital", "continent"]],
                                 villes[["id", "name"]].rename(
                                     columns={"id": "capital", "name": "capital_name"}),
                                 on="capital")

La liste des pays d’Océanie et leurs capitales s’obtient alors facilement :

In [ ]:
pays_et_capitales[pays_et_capitales.continent == 'OC']

## Bases de données
Comme pour l'opération de sélection, l'opération de jointure est une opération fondamentale des bases de données relationnelles

```sql
SELECT table1.colonne1, table1.colonne2, table2.colonne2
FROM table1, table2
WHERE table1.id = table2.id
```

## Exercices
On considère les tables suivantes
* Membre

| prénom  | idm |
|----------|-----|
| Bertrand | 12  |
| Lucie    | 24  |
| Jean     | 42  |
| Sara     | 7   |

* Prêt

| idl | idm |
|-----|-----|
| 12  | 42  |
| 11  | 7   |
| 3   | 42  |
| 4   | 7   |
| 8   | 12  |

* Livre

| idl | titre                 |
|-----|-----------------------|
| 1   | Fondation             |
| 2   | Les Robots            |
| 3   | Dune                  |
| 4   | La Stratégie Ender    |
| 5   | 1984                  |
| 6   | Santiago              |
| 7   | Hypérion              |
| 8   | Fahrenheit 451        |
| 9   | Les Monades urbaines  |
| 10  | Chroniques Martiennes |
| 11  | L’étoile et le fouet  |
| 12  | Ubik                  |

* Commandé

| idl | titre                  | livré |
|-----|------------------------|-------|
| 13  | Le Seigneur de lumière | Non   |
| 2   | Le Vagabond            | Oui   |
| 3   | Les Cavernes d’acier   | Oui   |

### Exercice 1
On considère les tables `Membre`, `Prêt` et `Livre`.  

Calculer à la main la jointure des tables `Membre` et `Prêt` et la jointure des tables `Prêt` et `Livre`.

### Exercice 2
Pour chacune des quatre tables, donner le contenu d'un fichier CSV correspondant.

### Exercice 3
Ecrire un programme Python qui charge les trois fichiers `membre.csv`, `pret.csv` et `livre.csv` et qui calcule la jointure de ces trois tables

### Exercice 4
Ecrire un programme Python qui charge les trois fichiers `membre.csv`, `pret.csv` et `livre.csv` comme des tables, qui calcule la jointure de ces trois tables et sauve le résultat dans un fichier.

### Exercice 5
La table `Commandé` liste les livres commandés et d'ils ont été livrés ou non.  
Que peut-on dire de ses colonnes par rapport à celles de `Livre` ?

Proposer une méthode permettant de faire la réunion des deux tables et la coder en Python

## Liens :
* Document accompagnement Eduscol : [Manipulation de tables avec la bibliothèque Pandas](https://cache.media.eduscol.education.fr/file/NSI/78/0/RA_Lycee_G_NSI_trtd_pandas_1170780.pdf)
* Jeus de données : [Prénoms à Rennes](https://data.rennesmetropole.fr/explore/dataset/prenoms-a-rennes/)
* CNAM : [Travaux pratiques Bases de données](https://deptfod.cnam.fr/bd/tp)
* W3School : [The Try-SQL Editor](https://www.w3schools.com/sql/trysql.asp?filename=trysql_op_in)